In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
import pandas as pd

In [2]:
dat = pd.read_csv("ectreetoExtract.csv", encoding="ISO-8859-1")

In [3]:
dat.head()

,id,SrNo,Classname,subclassof
0,677,1.1.1.1,carbon-monoxide dehydrogenase (cytochrome b-561),673


In [4]:
ectree_list = []

for i in range(len(dat)):
    if(len(dat['SrNo'][i]) >= 7):
        ectree_list.append(dat['SrNo'][i])

In [5]:
for ectree_str in range(len(ectree_list)):
    print(ectree_list[ectree_str])

1.1.1.1


In [6]:
for ectree_str in range(len(ectree_list)):
    driver = webdriver.Chrome()
    url = 'https://www.brenda-enzymes.org/enzyme.php?ecno=' + ectree_list[ectree_str]
    driver.get(url)
    #show all tables
    driver.find_element(By.XPATH, '//*[@id="flatnav_sa"]').click()
    title = driver.find_element(By.XPATH, '//*[@id="flatheader"]/div/h1')
    print(title.text)
    # //*[@id="tab47"] https://meet.google.com/csj-cxpp-ejv //*[@id="tab47"]
    try:
        title = driver.find_element(By.XPATH, '//*[@id="tab5_head"]/div[1]')
        print(title.text)
        chks = driver.find_elements(By.XPATH, "//*[@id='tab5']//a[@href]")

        ent = 'entries'

        for chk in chks:
            txt = chk.text
            if ent in txt:
                chk.click()
        tab1_eles = driver.find_elements(By.XPATH, '//div[@id="tab5" and @class="equal"]')
        table_columns = []

        for tab in tab1_eles:
            
            col_list = tab.find_elements(By.CLASS_NAME, 'header')
            
            for listss in col_list:
                # print(listss.text)
                table_columns.append(listss.text)
        final_df = pd.DataFrame(columns = table_columns)
        tab1_eles = driver.find_elements(By.XPATH, "//div[contains(@id, 'tab5r') and contains(@class, 'cell')]")
        table_values = []
        for tab in tab1_eles:
            try:
                link = tab.find_element(By.TAG_NAME, 'a')
                temp_link = link.get_attribute('href')
                
                # if not('showRows' or 'hideRows') in temp_link:
                if ('showRows' in temp_link or 'hideRows' in temp_link):
                    #print(tab.text)
                    table_values.append(tab.text)
                    # print(tab.text + ', ' + temp_link)
                    # table_values.append(tab.text + ',' + temp_link)
                    
                else:
                    # print(tab.text)
                    # table_values.append(tab.text)
                    
                    #print(tab.text + ', ' + temp_link)
                    table_values.append(tab.text + ',' + temp_link)
                    
            except:
                table_values.append(tab.text)
                #print(tab.text)
        for i in range(len(table_columns)):
            temp_value = table_values[i::(len(table_columns))]
            #print(temp_value)
            
            final_df[table_columns[i]] = temp_value
        final_df.to_csv('table_for_test.csv')
        final_df.replace(r'^\s*$', np.nan, regex=True, inplace = True)
        final_df.ffill(inplace = True)
        http_columns = final_df.columns[final_df.apply(lambda x: x.str.contains("https")).any()]
        # len(http_columns)
        # main_df = final_df.copy()
        for i in range(len(http_columns)):
            # print(http_columns[i])
            temp_string = http_columns[i] + ' ' + 'href'
            # new = final_df[http_columns[i]].str.split(",", n = 1, expand = True)
            new = final_df[http_columns[i]].str.rsplit(",", n=1, expand = True)
            # print(new[0])        
            # print(new[1])
            final_df[http_columns[i]] = new[0]
            final_df[temp_string]= new[1] 
        final_df.head(20)
        js_columns = final_df.columns[final_df.apply(lambda x: x.str.contains("javascript")).any()]
        new_columns = []

        for i in range(len(js_columns)):
            # print(js_columns[i])
            temp_string = js_columns[i] + ' ' + 'href'
            # new = final_df[js_columns[i]].str.split(",", n = 1, expand = True)
            new = final_df[js_columns[i]].str.rsplit(",", n = 1, expand = True)
                        
            final_df[js_columns[i]] = new[0]
            final_df[temp_string]= new[1] 
            
            new_columns.append(temp_string)

        # explode
        if ('LITERATURE' in final_df or 'UNIPROT' in final_df):
            final_df['LITERATURE'] = final_df['LITERATURE'].str.split(', ')
            final_df = final_df.explode(['LITERATURE'])
            
            final_df['UNIPROT'] = final_df['UNIPROT'].str.split(', ')
            final_df = final_df.explode(['UNIPROT'])
        final_df = final_df.reset_index()
        final_df.drop('index', axis = 1, inplace = True)
        # final_df.tail()
        # final_df['LITERATURE href'][0].split('&')[0] + '&r=' + final_df['LITERATURE'][0]
        # literature_temp = 'https://www.brenda-enzymes.org/literature.php?e=1.1.1.1&r=285624'
        final_df['LITERATURE href'] = "https://www.brenda-enzymes.org/literature.php?e=1.1.1.1&r=737159"
        for i in range(len(final_df)):
            try:
                # ectra codes
                if(final_df['LITERATURE'][i] != '-'):
                    final_df['LITERATURE href'][i] = final_df['LITERATURE href'][i].split('&')[0] + '&r=' + final_df['LITERATURE'][i]
                else:
                    final_df['LITERATURE href'][i] = '-'
            except:
                continue
        # final_df
        temp_list_href = []
        for i in range(len(new_columns)):
            temp_list = final_df[new_columns[i]]
            
            headers = [item for item in temp_list if item is not None]
            temp_list_href.append(headers[0])
        temp_list_href = temp_list_href[:2]
        resulted_link = ['https://www.brenda-enzymes.org/taxonomy.php?f[id][value]=1515','https://www.brenda-enzymes.org/sequences.php?AC=A3DCI2']
        temp_df = final_df[new_columns]
        for i in range(len(new_columns)):
            # print(new_columns[i])
            new_link = resulted_link[i]
            
            for j in range(len(temp_df)):
                old_link = temp_df[new_columns[i]].iloc[[j]].values
                
                try:
                    extracted_new_link_str = re.findall(r'\b\d+\b', new_link)
                    
                    extracted_old_link_str = re.findall(r"'(.*?)'", old_link[0], re.DOTALL) 
                    
                    if not extracted_new_link_str:
                        extracted_new_link_str = new_link.split("=",1)[1]
                        updated_final_link = new_link.replace(str(extracted_new_link_str), str(extracted_old_link_str[0]))
                    else:
                        updated_final_link = new_link.replace(str(extracted_new_link_str[0]), str(extracted_old_link_str[0]))
                        
                    # print(f'old link: {old_link}')
                    # print(f'extracted from old link: {extracted_old_link_str[0]}')
                    # print(f'new link: {new_link}')
                    # print(f'extracted from new link: {extracted_new_link_str[0]}') 
                    # 
                    # print(f'updated final link: {updated_final_link}')
                    # print('\n')
                    
                    temp_df[new_columns[i]].iloc[[j]]= updated_final_link
                
                except:
                    pass
        for i in range(len(new_columns)):
            final_df = final_df.drop(new_columns[i], axis = 1)
        final_df = final_df.drop_duplicates()
        diff = len(temp_df) - len(final_df)
        final_df.tail()
        temp_df.drop(temp_df.tail(diff).index,inplace = True)
        len(temp_df)
        frames = [final_df, temp_df]
        final_df = pd.concat([final_df, temp_df], axis = 1)
        final_df.head()
        final_df.index = np.arange(1, len(final_df) + 1)
        final_df.insert(loc = 0,column = 'id',value = final_df.index)
        final_df.insert(loc = 1,column = 'ectree',value = ectree_list[ectree_str])
        final_df.replace(to_replace=[None], value='-', inplace=True)
        final_df.columns = [c.replace(" ", "_") for c in final_df.columns]
        final_df.columns = [c.replace("(", "") for c in final_df.columns]
        final_df.columns = [c.replace(")", "") for c in final_df.columns]
        final_df.columns = [c.replace("?", "") for c in final_df.columns]
        final_df.columns = [c.replace("=", "_equals_") for c in final_df.columns]
        final_df = final_df.replace('', np.NaN)
        final_df.ffill(inplace = True)
        
        final_df['LITERATURE_href'] = final_df['LITERATURE_href'].replace("1.1.1.1", ectree_list[ectree_str], regex = True)
        #### reaction_diagram process
        driver.quit()
        #### Writing on mysql
        import mysql.connector
        mydb = mysql.connector.connect(host = 'quantumzyme.com', user = 'zummit_user', password = 'kZf-ih&[xV@2', database = 'qzBrenda', use_pure = True)
        # mydb = mysql.connector.connect(host = 'localhost', user = 'root', password = 'Darshini1@*',port = 3306, database = 'brenda', use_pure = True)
        
        #### create table string 
        final_df.drop('id', axis = 1, inplace = True)
        combined_string = []
        for i in range(len(final_df.columns)):
            if(len(final_df.columns[i]) <= 64):
                temp_string = final_df.columns[i] + ' VARCHAR(500)'
                combined_string.append(temp_string) 
            else:
                temp_string = final_df.columns[i][0:63] + ' VARCHAR(500)'
                combined_string.append(temp_string)
            
        combined_string_new = ', '.join(combined_string)
        #print(combined_string_new)
        #print('\n')

        create_table_string = 'CREATE TABLE IF NOT EXISTS '+ final_df.columns[1] + '(id int NOT NULL AUTO_INCREMENT, ' + combined_string_new + ', PRIMARY KEY (id))'
        #print(create_table_string)
        mycursor = mydb.cursor()
        try:
            mycursor.execute(create_table_string)

        except:
            pass
        #### insert into table
        combined_insert_string = []
        percent_s = []
        percent = '%s'
        for i in range(len(final_df.columns)):
            if(len(final_df.columns[i]) >= 64):
                combined_insert_string.append(final_df.columns[i][0:63])
                percent_s.append(percent)

            else:
                combined_insert_string.append(final_df.columns[i])
                percent_s.append(percent)

        #print(i)
        # percent_s = percent_s[:-1]
        percent_s = ', '.join(percent_s)

        combined_insert_string = ', '.join(combined_insert_string)

        insert_string = 'INSERT INTO ' + final_df.columns[1] + '(' + combined_insert_string + ' ) VALUES(' + percent_s  + ')'
        #print(insert_string)
        df_columns = []
        for i in range(len(final_df.columns)):
            if(len(final_df.columns[i]) >= 64):
                df_columns.append(final_df.columns[i][0:63])
            else:
                df_columns.append(final_df.columns[i]) 

        df_columns
        for(row, rs) in final_df.iterrows():
            for i in range(len(df_columns)):
                # print(rs[i])
                df_columns[i] = rs[i]
                    
            data = tuple(df_columns)
            mycursor.execute(insert_string, data)
            mydb.commit()
        mydb.close()

    except Exception as e:
        print('Exceptio2344n', repr(e))
        driver.quit()

Information on EC 1.1.1.1 - alcohol dehydrogenase
CRYSTALLIZATION (Commentary)


C:\Users\as787\AppData\Local\Temp\ipykernel_46464\1375651691.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[new_columns[i]].iloc[[j]]= updated_final_link
